In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
eve = input()
url = "https://www.vlr.gg/" + eve
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>
			CGN Esports vs. Acend | Mandatory.gg Cup #3 | Playoffs | Valorant match | VLR.gg		</title>
<meta content="Stats, score, streams, and VODs from CGN Esports vs. Acend - Grand Final match of Mandatory.gg Cup #3 Valorant event" name="description"/>
<meta charset="utf-8"/>
<meta content="CGN Esports vs. Acend | Mandatory.gg Cup #3 | Playoffs | Valorant match | VLR.gg" property="og:title"/>
<meta content="VLR.gg" property="og:site_name"/>
<meta content="Stats, score, streams, and VODs from CGN Esports vs. Acend - Grand Final match of Mandatory.gg Cup #3 Valorant event" property="og:description"/>
<meta content="/img/vlr/favicon.png" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="@vlrdotgg" name="twitter:site"/>
<meta content="CGN Esports vs. Acend | Mandatory.gg Cup #3 | Playoffs | Valorant match | VLR.gg" name="twitter:title"/>
<meta content="Stats, score, streams, and VODs from CGN Esports vs. Acend - G

- Avoir les matchs id = div -> class = 'vm-stats-game' puis data-game-id = 'all' ou 'xxxxxx'
- 'data-game-id' à récupérer => "vlr.gg/eve/?game='data-game-id'"
- &tab=overview/performance/economy
- Stats dans 'vm-stats-container'

Partie overview
- All maps.
- Then match per match.

All maps

In [4]:
all_maps = soup.find_all("div", {"class": "vm-stats-game"})
game_ids = []
for maps in all_maps:
    game_id = maps.get('data-game-id')
    game_ids.append(game_id)
maps

<div class="vm-stats-game" data-game-id="152028">
<div class="vm-stats-game-header">
<div class="team">
<div class="score" style="margin-right: 12px;">10 </div>
<div>
<div class="team-name">
							CGN Esports						</div>
<span class="mod-ct">7</span> /
						<span class="mod-t">3</span>
</div>
</div>
<div class="map">
<div style="font-weight: 700; font-size: 20px; text-align: center; position: relative; margin-bottom: 3px;">
<span style="position: relative;">
							Sunset														<span class="picked mod-1 ge-text-light">
								PICK
							</span>
</span>
</div>
<div class="map-duration ge-text-light" style="text-align: center;">
						
													52:56												
						
					

					</div>
</div>
<div class="team mod-right">
<div>
<div class="team-name">
							Acend						</div>
<span class="mod-t">5</span> /
						<span class="mod-ct">8</span>
</div>
<div class="score mod-win" style="margin-left: 8px;">13</div>
</div>
</div>
<div style="text-align: center; margin-top: 15px;

Dans map, deux élements par team.

In [5]:
maps.find_all("table", class_='wf-table-inset mod-overview')[1]

<table class="wf-table-inset mod-overview">
<thead>
<tr>
<th></th>
<th title="Agent"></th>
<th title="Rating"><span>R<i></i></span></th>
<th title="Average Combat Score"><span>ACS<i></i></span></th>
<th style="padding-left: 14px;" title="Kills"><span>K<i></i></span></th>
<th title="Deaths"><span>D<i></i></span></th>
<th title="Assists"><span>A<i></i></span></th>
<th title="Kills - Deaths"><span>+/–<i></i></span></th>
<th "="" title="Kill, Assist, Trade, Survive %"><span>KAST<i></i></span></th>
<th title="Average Damage per Round"><span>ADR<i></i></span></th>
<th style="padding-left: 1px;" title="Headshot %"><span>HS%<i></i></span></th>
<th style="padding-left: 9px;" title="First Kills"><span>FK<i></i></span></th>
<th title="First Deaths"><span>FD<i></i></span></th>
<th title="Kills - Deaths"><span>+/–<i></i></span></th>
</tr>
</thead>
<tbody>
<tr>
<td class="mod-player" style=" padding-right: 10px;">
<div style="display: flex; align-items: center;">
<i class="flag mod-ru" style="opacit

In [6]:
for e in maps.find_all("table", class_='wf-table-inset mod-overview'):
    team_1 = maps.find_all("table", class_='wf-table-inset mod-overview')[0]
    team_2 = maps.find_all("table", class_='wf-table-inset mod-overview')[1]

In [7]:
team_1

<table class="wf-table-inset mod-overview">
<thead>
<tr>
<th></th>
<th title="Agent"></th>
<th title="Rating"><span>R<i></i></span></th>
<th title="Average Combat Score"><span>ACS<i></i></span></th>
<th style="padding-left: 14px;" title="Kills"><span>K<i></i></span></th>
<th title="Deaths"><span>D<i></i></span></th>
<th title="Assists"><span>A<i></i></span></th>
<th title="Kills - Deaths"><span>+/–<i></i></span></th>
<th "="" title="Kill, Assist, Trade, Survive %"><span>KAST<i></i></span></th>
<th title="Average Damage per Round"><span>ADR<i></i></span></th>
<th style="padding-left: 1px;" title="Headshot %"><span>HS%<i></i></span></th>
<th style="padding-left: 9px;" title="First Kills"><span>FK<i></i></span></th>
<th title="First Deaths"><span>FD<i></i></span></th>
<th title="Kills - Deaths"><span>+/–<i></i></span></th>
</tr>
</thead>
<tbody>
<tr>
<td class="mod-player" style=" padding-right: 10px;">
<div style="display: flex; align-items: center;">
<i class="flag mod-de" style="opacit

In [38]:
stats_t1 = []
header_row = []
t_n = []
p_n = []
champions = []
num_columns = 12

for el in team_1.find_all('span', class_='mod-both'):
    stats_t1.append(el.get_text(strip=True))
table_data = [stats_t1[i:i + num_columns] for i in range(0, len(stats_t1), num_columns)]

for title in team_1.find_all('th'):
    header_row.append(title.get('title'))
header_row = list(filter(None, header_row))
header_row.remove('Agent')
for el in team_1.find_all('div', class_='ge-text-light'):
    t_n.append(el.get_text(strip=True))
for el in team_1.find_all('div', class_='text-of'):
    p_n.append(el.get_text(strip=True))
for el in team_1.find_all('span', class_='mod-agent'):
    champions.append(el.find('img').get('title'))
    
df_data = pd.DataFrame(table_data, columns=header_row)
df_champions = pd.DataFrame(champions, columns=['Agents'])
df_t_n = pd.DataFrame(t_n,columns=['Team'])
df_p_n = pd.DataFrame(p_n,columns=['Player'])
df = pd.concat([df_t_n, df_p_n, df_champions, df_data], axis=1)
df

,Team,Player,Agents,Rating,Average Combat Score,Kills,Deaths,Assists,Kills - Deaths,"Kill, Assist, Trade, Survive %",Average Damage per Round,Headshot %,First Kills,First Deaths,Kills - Deaths
0,CGN,Vince,Omen,1.21,257,20,16,8,+4,74%,159,33%,2,1,+1
1,CGN,BennY,Phoenix,1.17,231,21,16,2,+5,78%,154,34%,4,4,0
2,CGN,DaviH,Skye,1.03,151,12,14,6,-2,78%,107,32%,1,2,-1
3,CGN,aNguiSt,Cypher,0.74,151,13,17,3,-4,65%,112,64%,2,1,+1
4,CGN,elllement,Raze,0.58,124,11,20,2,-9,57%,84,29%,2,4,-2


In [36]:
header_row

['Agent',
 'Rating',
 'Average Combat Score',
 'Kills',
 'Deaths',
 'Assists',
 'Kills - Deaths',
 'Kill, Assist, Trade, Survive %',
 'Average Damage per Round',
 'Headshot %',
 'First Kills',
 'First Deaths',
 'Kills - Deaths']

In [39]:
stats_t2 = []
header_row = []
t_n = []
p_n = []
champions = []
num_columns = 12

for el in team_2.find_all('span', class_='mod-both'):
    stats_t2.append(el.get_text(strip=True))
table_data = [stats_t2[i:i + num_columns] for i in range(0, len(stats_t2), num_columns)]

for title in team_2.find_all('th'):
    header_row.append(title.get('title'))
header_row = list(filter(None, header_row))
header_row.remove('Agent')
for el in team_2.find_all('div', class_='ge-text-light'):
    t_n.append(el.get_text(strip=True))
for el in team_2.find_all('div', class_='text-of'):
    p_n.append(el.get_text(strip=True))
for el in team_2.find_all('span', class_='mod-agent'):
    champions.append(el.find('img').get('title'))
    
df_data = pd.DataFrame(table_data, columns=header_row)
df_champions = pd.DataFrame(champions, columns=['Agents'])
df_t_n = pd.DataFrame(t_n,columns=['Team'])
df_p_n = pd.DataFrame(p_n,columns=['Player'])
df2 = pd.concat([df_t_n, df_p_n, df_champions, df_data], axis=1)
df = pd.concat([df, df2], axis=0, ignore_index=True)
df

,Team,Player,Agents,Rating,Average Combat Score,Kills,Deaths,Assists,Kills - Deaths,"Kill, Assist, Trade, Survive %",Average Damage per Round,Headshot %,First Kills,First Deaths,Kills - Deaths
0,CGN,Vince,Omen,1.21,257,20,16,8,+4,74%,159,33%,2,1,+1
1,CGN,BennY,Phoenix,1.17,231,21,16,2,+5,78%,154,34%,4,4,0
2,CGN,DaviH,Skye,1.03,151,12,14,6,-2,78%,107,32%,1,2,-1
3,CGN,aNguiSt,Cypher,0.74,151,13,17,3,-4,65%,112,64%,2,1,+1
4,CGN,elllement,Raze,0.58,124,11,20,2,-9,57%,84,29%,2,4,-2
5,ACE,chiwa,Cypher,1.07,171,13,12,8,+1,78%,117,20%,0,0,0
6,ACE,baddyG,Viper,1.05,250,20,17,5,+3,74%,168,38%,4,3,+1
7,ACE,musashi,Gekko,1.00,215,18,16,3,+2,65%,136,27%,2,1,+1
8,ACE,ALIVE,Raze,0.98,219,19,17,2,+2,70%,141,17%,4,5,-1
9,ACE,Br0die,Omen,0.96,168,13,15,6,-2,87%,115,26%,2,2,0


In [ ]:
t_n = []
for el in team_1.find_all('div', class_='ge-text-light'):
    t_n.append(el.get_text(strip=True))
t_n

In [ ]:
p_n = []
for el in team_1.find_all('div', class_='text-of'):
    p_n.append(el.get_text(strip=True))
p_n

In [ ]:
champions = []
for el in team_1.find_all('span', class_='mod-agent'):
    champions.append(el.find('img').get('title'))
champions

In [ ]:
team_1.find_all('span', class_='mod-agent')[0].find('img').get('title')